In [1]:
import pandas as pd
import sqlite3

In [2]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/sql-murder-mystery.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [4]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

crime_scene_report
drivers_license
person
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
income
solution


In [6]:
query = '''
SELECT * 
FROM person
'''

df = sql_query(query)
df.head(10)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
5,10027,Antione Godbolt,439509,2431,Zelham Dr,491650087
6,10034,Kyra Buen,920494,1873,Sleigh Dr,332497972
7,10039,Francesco Agundez,278151,736,Buswell Dr,861079251
8,10095,Leslie Thate,729987,2772,Camellia Park Circle,127944356
9,10122,Alva Conkel,779002,116,Diversey Circle,148521773
